# Data
Data from simulations performed by McStasScript is returned as [McStasData](../_autosummary/mcstasscript.data.data.McStasData.rst) objects. This section will explore what these contain and how one can manipulate them. First a small instrument is written that will supply data to investigate.

## Example instrument
The instrument will consist of a source, a powder sample and some monitors that will record data.

In [ ]:
import mcstasscript as ms

instrument = ms.McStas_instr("data_example")

source = instrument.add_component("source", "Source_simple")
source.set_parameters(xwidth=0.05, yheight=0.03, dlambda=0.1,
                      dist=5, focus_xw=0.015, focus_yh=0.03)
source.lambda0 = instrument.add_parameter("wavelength", value=1.2)

sample = instrument.add_component("sample", "PowderN")
sample.set_parameters(radius=source.focus_xw, yheight=source.focus_yh,
                      reflections='"Na2Ca3Al2F14.laz"', barns=0)
sample.set_AT(source.dist, RELATIVE=source)

### Example monitors
Here three monitors are defined, a 2D PSD monitor, a 1D banana monitor and an event monitor. Monitor_nD is used for the last two, where the option string describes the geometry and what is to be recorded.

In [ ]:
banana = instrument.add_component("banana", "Monitor_nD", RELATIVE=sample)
banana.set_parameters(xwidth=1.5, yheight=0.4, restore_neutron=1, filename='"banana.dat"')
banana.options = '"theta limits=[5 175] bins=250, banana"'

In [ ]:
event = instrument.add_component("events", "Monitor_nD")
event.set_AT(0.1, RELATIVE=sample)
event.set_parameters(xwidth=0.1, yheight=0.1, restore_neutron=1, filename='"events.dat"')
event.options = '"list all auto, x y z vx vy vz t"'

In [ ]:
mon = instrument.add_component("PSD", "PSD_monitor")
mon.set_AT(0.1, RELATIVE=sample)
mon.set_parameters(nx=100, ny=100, filename='"psd.dat"',
                   xwidth=3*sample.radius, yheight=2*sample.yheight, restore_neutron=1)

## Generating data
The simulation is executed using the *backengine* method with a low number of neutrons. The data is returned by the *backengine* method, but will contain None if the simulation failed. 

In [ ]:
instrument.settings(ncount=1E5, output_path="data_example")
data = instrument.backengine()

In [ ]:
print(data)

## McStasData objects
The data retrieved from the instrument object is in the form of a list that contains [McStasDataBinned](../_autosummary/mcstasscript.data.data.McStasDataBinned.rst) and [McStasDataEvent](../_autosummary/mcstasscript.data.data.McStasDataEvent.rst) objects. McStasScript contains a function called [*name_search*](../_autosummary/mcstasscript.interface.functions.name_search.rst) which can be used to select a certain element of such a data list. It will match the component name first and if no match is found it will check for match with the filename. Here [*name_search*](../_autosummary/mcstasscript.interface.functions.name_search.rst) is used to retrieve the PSD [McStasDataBinned](../_autosummary/mcstasscript.data.data.McStasDataBinned.rst) object.

In [ ]:
PSD = ms.name_search("PSD", data)
banana = ms.name_search("banana", data)
events = ms.name_search("events", data)

### Accessing metadata
Both McStasDataBinned and McStasDataEvent object carries relevant metadata in a *metadata* attribute. Using the python print function this object can display basic information on the contained data.

In [ ]:
print(PSD.metadata)
print(banana.metadata)
print(events.metadata)

The metadata object has attributes which can be accessed as well. The info attribute is a dict with the raw metadata read from the file.

- component_name
- dimension
- filename
- limits
- parameters
- info

In [ ]:
PSD.metadata.info

The total intensity, error or ncount is kept in the values are often of interest, so these are given their own attributes: *total_I*, *total_E* and *total_N*

In [ ]:
print(PSD.metadata.total_I)
print(PSD.metadata.total_E)
print(PSD.metadata.total_N)

### Accessing the data
McStasData objects stores the data as [Numpy arrays](https://numpy.org/doc/stable/reference/generated/numpy.array.html), these can be accessed as attributes.

- Intensity: Holds the intensity, sum of all ray weights
- Error: Error on intensity
- Ncount: Number of rays that reached 

In [ ]:
print("Intensity")
print(PSD.Intensity)

print("Error")
print(PSD.Error)

print("Ncount")
print(PSD.Ncount)

The original path to the data is also contained within the McStasData object and can be returned with get_data_location.

In [ ]:
PSD.get_data_location()

### Event data
McStasDataEvent objecst stores event data, and for this reason does not have *Error* or *Ncount*. The event information is contained in a 2D Numpy array in the *Intensity* and *Events* attributes. 

In [ ]:
print(events)

In [ ]:
print("Events", events.Events)

The [McStasDataEvent](../_autosummary/mcstasscript.data.data.McStasDataEvent.rst) objects have some help functions to work with the contained event data. The first is *get_data_column* that returns the data for a given axis. The available axis shown in the table below.

| Axis string | Full name | Unit | Description |
| --- | --- | --- | --- |
| t | time | s | Particle time |
| x | x position | m | Coordinate x of particle |
| y | y position | m | Coordinate y of particle |
| z | z position | m | Coordinate z of particle |
| vx | x velocity | m/s | Velocity projected onto x |
| vy | y velocity | m/s | Velocity projected onto y |
| vz | z velocity | m/s | Velocity projected onto z |
| p | weight | intensity | Particle weight McStas n/s |
| l | lambda | AA | Wavelength |
| e | energy | meV | Particle energy |
| speed | speed | m/s | Speed (length of velocity vector) |
| dx | x divergence | deg | Divergence from z axis along x axis |
| dy | y divergence | deg | Divergence from z axis along y axis |
| U1 | User1 | Userdefined | User defined flag in Monitor_nD |
| U2 | User2 | Userdefined | User defined flag in Monitor_nD |
| U3 | User3 | Userdefined | User defined flag in Monitor_nD |


Here we get the wavelength information for each neutron in the event dataset.

In [ ]:
events.get_data_column("l")

It is also possible to produce a McStasDataBinned dataset from a McStasDataEvent dataset by binning along one or two chosen dimensions using *make_1d* and *make_2d*. It supports all the same variables as above. For each method it is possible to choose how many bins should be used with the n_bins keyword argument.

In [ ]:
speed_1d = events.make_1d("speed", n_bins=60)
divergence_2d = events.make_2d("dx", "dy", n_bins=[120, 120])

print(speed_1d)
print(divergence_2d)

### Plotting 
[McStasData](../_autosummary/mcstasscript.data.data.McStasData.rst) objects contain information on how the data should be plotted, including for example if it should be on a logarithmic axis. This information is contained in the *plot_options* attribute of a [McStasData](../_autosummary/mcstasscript.data.data.McStasData.rst) object. The plotting are described in more detail on the [plotting page](plotting.ipynb).

In [ ]:
PSD.plot_options

McStasScript can plot a McStasData object directly using for example *make_plot*.

In [ ]:
ms.make_plot(PSD)

The *plot_options* can be updated with *set_plot_options* that takes keyword arguments.

In [ ]:
PSD.set_plot_options(log=True, top_lim=1.5, bottom_lim=-1.5, colormap="hot", orders_of_mag=2)
ms.make_plot(PSD)

The *set_plot_options* takes the following keyword arguments. Some will only apply for 2D data, for example *orders_of_mag*.

| Keyword argument | Type | Default | Description |
| --- | --- | --- | --- |
| log | bool | False | Logarithmic axis for y in 1D or z in 2D |
| orders_of_mag | float | 300 | Maximum orders of magnitude to plot in 2D |
| colormap | str | "jet" | Matplotlib colormap to use |
| show_colorbar | bool | True | Show the colorbar |
| x_axis_multiplier | float | 1 | Multiplier for x axis data |
| y_axis_multiplier | float | 1 | Multiplier for y axis data |
| cut_min | float | 0 | Unitless lower limit normalized to data range |
| cut_max | float | 1 | Unitless upper limit normalized to data range |
| left_lim | float | | Lower limit to plot range of x axis |
| right_lim | float | | Upper limit to plot range of x axis|
| bottom_lim | float | | Lower limit to plot range of y axis|
| top_lim | float | | Upper limit to plot range of y axis|

McStasDataBinned generated from Event files can be plotted in the same manner.

In [ ]:
ms.make_sub_plot([speed_1d, divergence_2d], log=True)